In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, Layout, Label, Box, VBox, HBox, Button
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import os 
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
import time
%matplotlib widget

In [ ]:
# Thermodynamic and other constants
T0 = 273.15 # melting point of water (K)
Rv = 461.5 # water vapor gas constant (J/kg/K)
Ls = 2.83e6 # latent heat of sublimation at 0 deg C (J/kg)
Lv = 2.501e6 # latent heat of vaporization at 0 deg C (J/kg)
cp = 1005 # specific heat at constant pressure for dry air (J/kg)
Rd = 287 # dry air gas constant (J/kg)
eps = Rd / Rv # "epsilon" = 0.622
rhol = 1000. # density of liquid water (kg/m^3)
p0 = 100000. # Reference pressure (Pa)
es0 = 611. # saturation vapor pressure at the triple point of water

Mw = 0.01801528 # kg/mol
sigma = 0.075 # N/m
Rv = 461.5 # J/kg/K
rhol = 1000. # kg/m^3
m_solute = 1.e-19 # kg

solute_dict = {
    'sodium chloride': {'molecular_weight': 0.0584, 'dissociation': 2},
    'ammonium sulfate': {'molecular_weight': 0.1321, 'dissociation': 3},
    'magnesium chloride': {'molecular_weight': 0.0952, 'dissociation': 3}
}

Ms_sc = 0.0584 # kg/mol
Ms_as = 0.1321 # kg/mol
Ms_mc = 0.0952 # kg/mol
i_sc = 2 
i_as = 3
i_mc = 3

In [ ]:
# Function and class definitions
class InteractiveKohlerPlot:
    def __init__(self):
        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.mass_solute = 1.e-19  # Default value (kg)
        self.temperature = 280.  # Default value (K)
        self.pressure = 1000. # Default value (hPa)
        self.solute_type = 'sodium chloride'
        self.ambient_sat_ratio = 1.
        self.ambient_line = None
        self.radii_mum = np.logspace(-2,1,100) # micrometers
        self.radii_m = self.radii_mum * 1.e-6 # meters
        self.solute_dict = {
            'sodium chloride': {'molecular_weight': 0.0584, 'dissociation': 2},
            'ammonium sulfate': {'molecular_weight': 0.1321, 'dissociation': 3},
            'magnesium chloride': {'molecular_weight': 0.0952, 'dissociation': 3}
        }
        self.last_point = None
        # self.anim_point, = self.ax.plot([], [], color='purple', marker='o', ms=10, animated=True)  # Prepare animated point
        self.anim = None  # Placeholder for the animation object
        self.dt = 0.001 # Initial time step for solution drop animation (s)
        self.time = 0 # For keeping track of total time (s)
        
        # Create and store widgets as attributes
        self.mass_solute_slider = widgets.FloatSlider(
            value=self.mass_solute, min=0.01*1.0e-19, max=100.0*1.0e-19, 
            step=0.01*1.0e-19, readout_format='.2e', continuous_update=True
        )
        self.temperature_slider = widgets.FloatSlider(
            value=self.temperature, min=250.0, max=350.0, step=0.1, 
            readout_format='.2f', continuous_update=True
        )
        self.solute_type_dropdown = widgets.Dropdown(
            options=['sodium chloride', 'ammonium sulfate', 'magnesium chloride'], 
            value=self.solute_type
        )
        self.ambient_sat_ratio_slider = widgets.FloatSlider(
            value=self.ambient_sat_ratio, min=0.995, max=1.006, step=0.0001, 
            readout_format='.4f', continuous_update=True
        )

        # Buttons for adjusting time step
        self.increase_time_step_btn = Button(description='Increase Δt (x10)')
        self.decrease_time_step_btn = Button(description='Decrease Δt (x10)')
        
        # Use the widgets in HBox layouts with labels for alignment
        mass_solute_widget = HBox([Label('Mass Solute (kg):', layout=Layout(width='150px')), self.mass_solute_slider])
        temperature_widget = HBox([Label('Temperature (K):', layout=Layout(width='150px')), self.temperature_slider])
        solute_type_widget = HBox([Label('Solute Type:', layout=Layout(width='150px')), self.solute_type_dropdown])
        ambient_sat_ratio_widget = HBox([Label('Ambient Saturation Ratio:', layout=Layout(width='150px')), self.ambient_sat_ratio_slider])

        # Arrange buttons horizontally
        time_step_buttons = HBox([self.increase_time_step_btn, self.decrease_time_step_btn])
        
        # Organize the HBoxes in a VBox layout
        self.widgets_layout = VBox([solute_type_widget, mass_solute_widget, temperature_widget, ambient_sat_ratio_widget, time_step_buttons],
                                   layout=Layout(padding='10px', margin='10px'))
        
        # Widget event handlers
        self.mass_solute_slider.observe(self.on_mass_solute_change, 'value')
        self.temperature_slider.observe(self.on_temperature_change, 'value')
        self.solute_type_dropdown.observe(self.on_solute_type_change, 'value')
        self.ambient_sat_ratio_slider.observe(self.on_ambient_sat_ratio_change, 'value')
        # Attach click event handlers to the buttons
        self.increase_time_step_btn.on_click(self.increase_time_step)
        self.decrease_time_step_btn.on_click(self.decrease_time_step)

        # Connect the onclick event (for creating a solution drop at a given radius)
        self.cid = self.fig.canvas.mpl_connect('button_press_event', self.onclick)

        # Initial plot setup
        self.update_plot()

    def increase_time_step(self, btn):
        self.dt *= 10
        # Additional code to handle any necessary updates when the time step changes
        self.time_annotation.set_text(f'Time step, total time: {self.dt}, {self.time:.2f} s')

    def decrease_time_step(self, btn):
        self.dt /= 10
        # Additional code to handle any necessary updates when the time step changes
        self.time_annotation.set_text(f'Time step, total time: {self.dt}, {self.time:.2f} s')
    
    def display_widgets(self):
        # Display the organized widgets
        display(self.widgets_layout)
    
    def update_plot(self): 
        # Clear and redraw the plot
        self.ax.clear()
        chosen_solute = self.solute_dict[self.solute_type]
        molecular_weight = chosen_solute['molecular_weight']
        dissociation = chosen_solute['dissociation']
        kohler_sat_ratio = self.calculate_kohler_curve(self.radii_m)
        plot_kwargs = dict(color='b', alpha=0.5, label=f'{self.solute_type}, m={self.mass_solute:.2e} kg')
        self.ax.semilogx(self.radii_mum, kohler_sat_ratio, **plot_kwargs)
        self.ambient_line = self.ax.axhline(self.ambient_sat_ratio, color='r', linestyle='--', label=f'$S$ = {self.ambient_sat_ratio:.3f}')
        self.ax.set_xlim(1.e-2, 1.e1)
        self.ax.set_ylim(0.995, 1.006)
        self.ax.set_xlabel(r'radius $\mu m$')
        self.ax.set_ylabel(r'saturation ratio')
        self.ax.legend(loc='best')
        self.time_annotation = self.ax.annotate(f'Time step, total time: {self.dt}, {self.time:.2f} s', xy=(0.05, 0.95), 
                                                xycoords='axes fraction')
        self.fig.canvas.draw_idle()

    def calc_es(self):
        """Calculates saturation vapor pressure with respect to water as a function of T"""
        return 611. * np.exp((Lv / Rv) * (1. / T0 - 1. / self.temperature))
    
    def calculate_kohler_curve(self, radii):
        """Exact form of Kohler curve"""
        chosen_solute = self.solute_dict[self.solute_type]
        i = chosen_solute['dissociation']
        m = self.mass_solute
        Ms = chosen_solute['molecular_weight']
        
        term1 = np.exp(2.*sigma/(radii*Rv*rhol*self.temperature))
        term2 = (1.+i*m*Mw/(Ms*4./3.*np.pi*radii**3.*rhol))**-1
        return term1*term2

    def calculate_Fk(self):
        K = self.calc_K()
        return (Lv/(Rv*self.temperature)-1)*(Lv*rhol/(K*self.temperature))

    def calculate_Fd(self):
        D = self.calc_D()
        es = self.calc_es()
        return (rhol*Rv*self.temperature)/(D*es)
    
    def calculate_DGE(self, radius):
        """Computes diffusional growth equation including Kohler effects. Returns growth (or evap) rate in m/s"""
        Fk = self.calculate_Fk()
        Fd = self.calculate_Fd()
        X = self.calculate_kohler_curve(radius)
        return (self.ambient_sat_ratio - X) / (radius*(Fk + Fd)) 
    
    def calc_K(self):
        """Calculates air thermal conductivity as a function of T"""
        return 2.4e-2 + (self.temperature - T0)*8.e-5

    def calc_D(self):
        """Calculates water vapor diffusion coefficient as a function of T and p, where p is in units of 
           hPa and T is in units of K"""
        return (1000. / self.pressure) * (2.21e-5 + (self.temperature - T0) * 1.5e-7)
    
    def on_mass_solute_change(self, change):
        self.mass_solute = change['new']
        self.update_plot()
    
    def on_temperature_change(self, change):
        self.temperature = change['new']
        self.update_plot()

    def on_solute_type_change(self, change):
        self.solute_type = change['new']
        self.update_plot()

    def on_ambient_sat_ratio_change(self, change):
        self.ambient_sat_ratio = change['new']
        # Update only the ambient saturation ratio line, not the entire plot
        self.update_ambient_sat_ratio_line()
        # Check the solution drop's equilibrium saturation ratio with the ambient again and trigger a new animation if needed
        if not self.anim:
            x, y = self.last_point.get_data()
            # new_log_entry = f"inside on_ambient_sat_ratio_change to animate drop {x}, {y}"
            # log_widget.value += new_log_entry  # Append the new log entry
            # Since get_data() returns arrays, extract the first element if they contain only one element
            x = x[0] if len(x) > 0 else 0
            y = y[0] if len(y) > 0 else 0
            self.start_animation(x, y)
        # self.update_plot()

    def update_ambient_sat_ratio_line(self):
        # Remove the old ambient saturation ratio line if it exists
        if hasattr(self, 'ambient_line'):
            self.ambient_line.remove()
    
        # Draw a new horizontal line at the updated ambient saturation ratio
        # self.ambient_line = self.ax.axhline(self.ambient_sat_ratio, color='r', linestyle='--')
        self.ambient_line = self.ax.axhline(self.ambient_sat_ratio, color='r', linestyle='--', label=f'$S$ = {self.ambient_sat_ratio:.3f}')

        # Update the legend
        # First, remove the old legend if it exists
        if self.ax.get_legend():
            self.ax.get_legend().remove()
        
        # Create a new legend that includes all the plot elements
        # You might need to gather all the labels and handles again, depending on your plot setup
        handles, labels = self.ax.get_legend_handles_labels()
        self.ax.legend(handles, labels, loc='best')
        
        # Redraw the canvas to reflect the updated line
        self.fig.canvas.draw_idle()
       
    def onclick(self, event):
        # Use the current state parameters for handling the click
        radius = event.xdata * 1.e-6
        # Log to the text widget
        # new_log_entry = f"Clicked coordinates: x={event.xdata}, y={event.ydata}\n"
        # log_widget.value += new_log_entry  # Append the new log entry
        if radius:
            # Set time to zero
            self.time = 0
            # If there's a last_point plotted, remove it from the axes
            if self.last_point:
                self.last_point.remove()
            start_saturation_ratio = self.calculate_kohler_curve(np.array([radius]))[0]
            # new_log_entry = f"Clicked coordinates: x={radius}, y={start_saturation_ratio}\n"
            # log_widget.value += new_log_entry  # Append the new log entry
            # Plot the new point and store the reference to it in last_point
            self.last_point, = self.ax.plot(event.xdata, start_saturation_ratio, color='purple', marker='o', ms=10)
            
            # Now compare with the ambient saturation ratio and slowly move the point depending on whether we are supersaturated
            # or subsaturated

            if self.anim:
                self.anim.event_source.stop()  # Stop any existing animation
            self.start_animation(event.xdata, start_saturation_ratio)
                

    def start_animation(self, start_x, start_saturation_ratio):
        # This function animates a point from start_x towards the intersection

        # new_log_entry = f"inside start_animation!"
        # log_widget.value += new_log_entry  # Append the new log entry
        
        def init():
            # new_log_entry = f"inside init!"
            # log_widget.value += new_log_entry  # Append the new log entry
            self.last_point.set_data([start_x], [start_saturation_ratio])
            # new_log_entry = f"start_x, start_y: {start_x}, {start_saturation_ratio}\n"
            # log_widget.value += new_log_entry  # Append the new log entry
            return self.last_point,

        def animate(i):
            # new_log_entry = f"inside animate!"
            # log_widget.value += new_log_entry  # Append the new log entry
            # Update the animated point's position
            x, y = self.last_point.get_data()
            # new_log_entry = f"inside animate 2! {x}, {y}"
            # log_widget.value += new_log_entry  # Append the new log entry
            # Since get_data() returns arrays, extract the first element if they contain only one element
            x = x[0] if len(x) > 0 else 0
            y = y[0] if len(y) > 0 else 0

            # new_log_entry = f"x, y: {x}, {y}\n"
            # log_widget.value += new_log_entry  # Append the new log entry

            diff_S = self.ambient_sat_ratio - y
            # Compute growth/evap rate from DGE
            drdt = self.calculate_DGE(np.array([x * 1.e-6]))[0] # m/s
            drdt = drdt * 1.e6 # Get to micrometers/s

            dx = drdt * self.dt
            # If dx becomes too large (more than 10% of the current radius), reduce the time step by a factor of 10 and try again.
            while np.abs(dx/x) > 0.1:
                self.dt = self.dt / 10.
                dx = drdt * self.dt
            self.time = self.time + self.dt  
            # Update time readout annotation
            self.time_annotation.set_text(f'Time step, total time: {self.dt}, {self.time:.2f} s')
            
            # new_log_entry = f"drdt, dr, radius, total time, {drdt}, {dx}, {x}, {self.time}\n"
            # log_widget.value += new_log_entry  # Append the new log entry
            
            # Calculate the new radius and equilibrium saturation ratio of the solution drop along its Kohler curve
            new_x = x + dx 
            new_y = self.calculate_kohler_curve(np.array([new_x * 1.e-6]))[0]
            self.last_point.set_data([new_x], [new_y])

            # new_log_entry = f"new_x, new_y: {new_x}, {new_y}\n"
            # log_widget.value += new_log_entry  # Append the new log entry

            # Check if the solution drop's equilibrium S is close enough to ambient, and stop the animation if so.
            if np.abs(diff_S) < 0.0001:
                self.time = 0.  # Reset time to zero
                self.anim.event_source.stop()  # Stop the animation if close enough
                self.anim = None

            # Stop the animation when the drop grows beyond 10 mum
            if new_x > 10.:
                self.time = 0.  # Reset time to zero
                # Reset it to 10 mum so that it can still animate (evaporate) later if the saturation ratio drops below the equilibrium value
                new_x = 10.
                new_y = self.calculate_kohler_curve(np.array([new_x * 1.e-6]))[0]
                self.last_point.set_data([new_x], [new_y])
                self.anim.event_source.stop()
                self.anim = None
                
            return self.last_point,

        self.anim = FuncAnimation(self.fig, animate, init_func=init, interval=20, blit=True)
        return self.anim,

In [ ]:
%%html
<style>
.jupyter-widgets.widget-label.jupyter-matplotlib-header {
    display: none;
}
</style>

In [ ]:
# Create an instance of the interactive plot

# Create a text widget to act as a log
# log_widget = widgets.Textarea(
#     value='',
#     placeholder='Log output will appear here...',
#     description='Log:',
#     disabled=False,
#     layout={'width': '100%', 'height': '200px'}
# )


interactive_plot = InteractiveKohlerPlot()
# Display the widgets
interactive_plot.display_widgets()
# display(log_widget)